# Worksheet 5-1: Nonlinear Regression

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WCC-Engineering/ENGR240/blob/main/Class%20Demos%20and%20Activities/Week%205/Worksheet%205-1_template%20nonlinear%20regression%20(simplified).ipynb)

## Overview

In this worksheet, we'll explore three different approaches to fit nonlinear models to data:

1. **Linear regression on transformed data**: Converting a nonlinear model to linear form
2. **Nonlinear regression using scipy.optimize.minimize**: Direct optimization of sum of squared residuals
3. **Nonlinear regression using scipy.optimize.curve_fit**: A more convenient wrapper for nonlinear regression

We'll apply these techniques to a bacterial growth rate model and compare their results.

## The Model

We'll work with a bacterial growth rate model that describes how growth rate depends on substrate concentration:

$$k = k_{max} \frac{c^2}{c_s + c^2}$$

where:
- $k$ is the growth rate (number/day)
- $c$ is the substrate concentration (mg/L)
- $k_{max}$ is the maximum possible growth rate
- $c_s$ is the half-saturation constant

In [ ]:
# Import necessary libraries
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

## Experimental Data and Model Function

Let's load our experimental data and define our model function:

In [ ]:
# Substrate concentration (mg/L)
c = np.array([0.5, 0.8, 1.5, 2.5, 4.0])

# Growth rate (number/day)
k = np.array([1.0, 2.5, 5.1, 7.3, 9.1])

# Plot the experimental data
plt.figure(figsize=(8, 5))
plt.scatter(c, k, color='black', s=50, label='Experimental data')
plt.xlabel('Substrate Concentration (mg/L)')
plt.ylabel('Growth Rate (number/day)')
plt.title('Bacterial Growth Rate vs. Substrate Concentration')
plt.legend()
plt.grid(True)
plt.show()

# Define model function
def kmodel(c, kmax, cs):
    """Bacterial growth rate model
    
    Parameters:
    -----------
    c : array_like
        Substrate concentration (mg/L)
    kmax : float
        Maximum growth rate (number/day)
    cs : float
        Half-saturation constant
    """
    return kmax * c**2 / (cs + c**2)

## Helper Functions

Let's define a function to calculate fit quality metrics that we'll use for all methods:

In [ ]:
def calculate_fit_metrics(c_data, k_data, kmax, cs, method_name):
    """Calculate and print fit quality metrics
    
    Parameters:
    -----------
    c_data : array_like
        Concentration data
    k_data : array_like
        Growth rate data
    kmax : float
        Fitted maximum growth rate
    cs : float
        Fitted half-saturation constant
    method_name : str
        Name of the fitting method
        
    Returns:
    --------
    tuple : (r_squared, Syx)
        Coefficient of determination and standard error
    """
    # Calculate predicted values
    k_pred = kmodel(c_data, kmax, cs)
    
    # Calculate residuals
    residuals = k_data - k_pred
    
    # Sum of Squared Residuals (Sr)
    Sr = np.sum(residuals**2)
    
    # Total Sum of Squares (St)
    St = np.sum((k_data - np.mean(k_data))**2)
    
    # Coefficient of Determination (R²)
    r_squared = 1 - Sr/St
    
    # Standard Error of the Estimate (Syx)
    Syx = np.sqrt(Sr/(len(k_data)-2))
    
    print(f"\nFit Quality Metrics for {method_name}:")
    print(f"kmax = {kmax:.4f}")
    print(f"cs = {cs:.4f}")
    print(f"R² = {r_squared:.4f}")
    print(f"Syx = {Syx:.4f}")
    
    return r_squared, Syx

## Task 1: Linear Regression with Transformed Data

We can transform our nonlinear model into a linear form by taking the reciprocal of both sides:

$$\frac{1}{k} = \frac{c_s + c^2}{k_{max} \cdot c^2} = \frac{c_s}{k_{max} \cdot c^2} + \frac{1}{k_{max}}$$

This gives us the form $Y = mX + b$ where:
- $Y = \frac{1}{k}$
- $X = \frac{1}{c^2}$
- $m = \frac{c_s}{k_{max}}$
- $b = \frac{1}{k_{max}}$

Let's implement this linear regression approach:

In [ ]:
# Transform the data
X = 1/c**2  # X = 1/c²
Y = 1/k     # Y = 1/k

# Perform linear regression using np.polyfit
p = np.polyfit(X, Y, 1)
m, b = p

# Calculate kmax and cs from m and b
kmax1 = 1/b
cs1 = m * kmax1

# Calculate fit metrics
r_squared1, Syx_1 = calculate_fit_metrics(c, k, kmax1, cs1, "Linear Regression (transformed)")

## Task 2: Nonlinear Regression using scipy.optimize.minimize

Instead of transforming our model, we can directly fit the nonlinear model to the data using optimization techniques. We'll define an objective function that calculates the sum of squared residuals (Sr) and use `scipy.optimize.minimize` to find the parameters that minimize this value:

In [ ]:
# Define the objective function (sum of squared residuals)
def objective_function(params, c_data, k_data):
    kmax, cs = params
    k_pred = kmodel(c_data, kmax, cs)
    residuals = k_data - k_pred
    return np.sum(residuals**2)

# Initial guess for parameters [kmax, cs]
initial_guess = [1, 1]

# Perform the optimization
result = optimize.minimize(objective_function, 
                          initial_guess, 
                          args=(c, k), 
                          method='Nelder-Mead',
                          tol=1e-8)

# Extract the optimized parameters
kmax2, cs2 = result.x

# Calculate fit metrics
r_squared2, Syx_2 = calculate_fit_metrics(c, k, kmax2, cs2, "Nonlinear Regression (minimize)")

## Task 3: Nonlinear Regression using scipy.optimize.curve_fit

The `scipy.optimize.curve_fit` function provides a more convenient interface for fitting models to data. Under the hood, it uses optimization techniques similar to `scipy.optimize.minimize` but with a more user-friendly API specifically designed for curve fitting:

In [ ]:
# Perform curve fitting
popt, pcov = optimize.curve_fit(kmodel, c, k, p0=[1, 1])

# Extract the optimized parameters
kmax3, cs3 = popt

# Extract parameter uncertainties
parameter_variances = np.diag(pcov)
parameter_std_dev = np.sqrt(parameter_variances)

# Calculate fit metrics
r_squared3, Syx_3 = calculate_fit_metrics(c, k, kmax3, cs3, "Nonlinear Regression (curve_fit)")

# Print the parameter uncertainties
print(f"\nParameter Uncertainties (Standard Deviations):")
print(f"kmax = {kmax3:.4f} ± {parameter_std_dev[0]:.4f}")
print(f"cs = {cs3:.4f} ± {parameter_std_dev[1]:.4f}")

## Task 4: Compare All Three Methods

Let's visualize and compare the results of all three fitting methods on a single plot and in a summary table:

In [ ]:
# Create a comparison table
methods = ['Linear Regression (transformed)', 'Nonlinear Regression (minimize)', 'Nonlinear Regression (curve_fit)']
kmax_values = [kmax1, kmax2, kmax3]
cs_values = [cs1, cs2, cs3]
r_squared_values = [r_squared1, r_squared2, r_squared3]
syx_values = [Syx_1, Syx_2, Syx_3]

# Print the comparison table
print("Comparison of Curve Fitting Methods:")
print("-" * 80)
print(f"{'Method':<30} {'kmax':<10} {'cs':<10} {'R²':<10} {'Syx':<10}")
print("-" * 80)
for i, method in enumerate(methods):
    print(f"{method:<30} {kmax_values[i]:<10.4f} {cs_values[i]:<10.4f} {r_squared_values[i]:<10.4f} {syx_values[i]:<10.4f}")
print("-" * 80)

# Plot the comparison
c_plot = np.linspace(0, 5, 100)
plt.figure(figsize=(10, 6))
plt.scatter(c, k, color='black', s=60, label='Experimental data')

# Plot the model fits
plt.plot(c_plot, kmodel(c_plot, kmax1, cs1), 'r-', linewidth=2, 
         label=f'Linear regression: R²={r_squared1:.4f}')
plt.plot(c_plot, kmodel(c_plot, kmax2, cs2), 'b--', linewidth=2, 
         label=f'Nonlinear (minimize): R²={r_squared2:.4f}')
plt.plot(c_plot, kmodel(c_plot, kmax3, cs3), 'g-.', linewidth=2, 
         label=f'Nonlinear (curve_fit): R²={r_squared3:.4f}')

plt.xlabel('Substrate Concentration (mg/L)', fontsize=12)
plt.ylabel('Growth Rate (number/day)', fontsize=12)
plt.title('Comparison of Three Curve Fitting Approaches', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True)
plt.tight_layout()
plt.show()

## Task 5: Discussion Questions

Based on your observations, answer the following questions:

1. **Method Comparison**: How do the results of the three methods compare? Which method(s) produced the best fit (highest R², lowest Syx)? Why might this be the case?

2. **Linear vs. Nonlinear Approaches**: What are the potential limitations or drawbacks of using a linear transformation for nonlinear models? Why might the nonlinear methods perform better in this case?

3. **Curve_fit vs. Minimize**: What are the advantages of using `curve_fit` over `minimize`? Note that only the `curve_fit` method provided parameter uncertainty estimates - why is this information valuable?

## Relationship Between curve_fit and minimize

You may have noticed that methods 2 and 3 (minimize and curve_fit) produced identical or nearly identical results. This is because `curve_fit` is essentially a wrapper function that uses optimization techniques similar to `minimize` under the hood, but with additional features:

- `curve_fit` is specifically designed for curve fitting problems with a simpler interface
- It automatically provides parameter uncertainty estimates (covariance matrix)
- It handles array inputs and outputs more conveniently
- It uses the Levenberg-Marquardt algorithm by default (a specialized algorithm for nonlinear least squares)

However, `minimize` offers more flexibility for complex optimization problems beyond curve fitting, such as constrained optimization, different optimization algorithms, and custom objective functions.

## Application to Real-World Problems

Nonlinear regression is used extensively in various fields of science and engineering. For example:

- **Michaelis-Menten Enzyme Kinetics**: $v = \frac{V_{max} \cdot [S]}{K_m + [S]}$ (reaction velocity vs. substrate concentration)
- **Exponential Decay**: $N(t) = N_0 e^{-\lambda t}$ (radioactive decay, population decline)
- **Sigmoidal Dose-Response**: $R = \frac{R_{max}}{1 + (\frac{EC_{50}}{C})^n}$ (drug response vs. concentration)

For these types of models, nonlinear regression techniques like `curve_fit` are often preferred for their accuracy and ability to provide parameter uncertainty estimates.